# So I've tried doing some of the tutorials and am having trouble producing a structure I like - what should I do!

This can depend quite a bit on what type of system you are looking for and what type of errors you are okay with.

This document is intended to be a living document with actual examples where debugging is/was necessary - along with what was done to resolve.

Generally each cell represents a different type of error and lists inside each cell are ordered by frequency of issues.

# 1 - No structures were produced!

This is quite common when looking at more complex systems and may be due to a number of factors.

## a. Unphysical input

Problem: Architector will try and build whatever you give it - sometimes for highly-coordinated systems this can result in overcrowding of the metal center and overlapping atoms. 
Solution: Double check your input and ligand definitions - we have had trouble getting this to work in cases where the metal-coordinating atoms were mislabelled, resulting in Architector attemping unphysical ligand conformations.

## b. XTB dependency

Problem: Most of the time Architector will use XTB (GFN2-xTB) as a backend for evaluation of energies, structures, and forces. Occassionally, XTB will not perform well on systems resulting in consistent compression of bond lengths, or very strange relative conformer energetic rankings.

Solutions:
1. Switch "relax" parameter to False. This will stop geometric deviations resulting from xTB relaxation - but will also not allow for an xTB relaxation meaning your produced conformer will often be more distorted at the end.
2. Check xTB convergence parameters. "xtb_electronic_temperature" can be set to higher (e.g. 1000K) to encourage convergence. Adding a solvent (e.g. "sovlent":"water") is another way to encourage more physical conformers because it screens the surface of ligand molecules forcing them to be closer to the metal in most cases.
3. Switch the methods involved in molcule construction. "assemble_method":"GFN-FF" or "assemble_method":"UFF" will be much faster but less accurate than GFN2-xTB. These are both viable parameters for the "full_method" key as well.

## c. Conformer rankings ruling out sane geometries

Problem: Architector defaults are limiting the scope of metal center symmetries sampled so that the "correct" one is not there.

Solution: Increase "n_conformers" to relax additional metal center symmetries at each core geometry. As additional explanation of these parameters: Increasing "n_symmetries" requests architector to attempt to build additional metal center symmetries by relative ligand postions. "n_conformers", however, specifies the number of generated "n_symmetries" that should relaxed with the "full_method" for each metal center. Note that n_symmetries should always be greater than or equal to n_conformers.

As a more concrete example of n_symmetries vs. n_conformers - play around with the following 3 examples:
1. n_symmetries:1 and n_conformers:1:
```python
from architector import (build_complex, 
                         view_structures)
out = build_complex({'core':{'metal':'Fe','coreCN':6},
                     'ligands':['water']*3+['chloride','bromide','iodide'],
                     'parameters':{'n_symmetries':1,'n_conformers':1,'skip_duplicate_tests':True,'relax':False}
                    })
view_structures(out)
```
This should produce 4 structures - one for each metal center symmetry: octahedral, pentagonal_pyramidal, trigonal_prismatic, and hexagonal planar. The 'skip_duplicate_tests' and 'relax':False keywords force Architector to return all structures regardless if they converge to sane geometries at the GFN2-xTB level of theory or create nearly identical structures.

2. n_symmetries:2 and n_conformers:1:
```python
out = build_complex({'core':{'metal':'Fe','coreCN':6},
                     'ligands':['water']*3+['chloride','bromide','iodide'],
                     'parameters':{'n_symmetries':2,'n_conformers':1,'skip_duplicate_tests':True,'relax':False}
                    })
view_structures(out)
```
This should produce 4 structures again - one for each metal center symmetry: octahedral, pentagonal_pyramidal, trigonal_prismatic, and hexagonal planar. But will potential produce some structures with different ligand symmetries than before (it did for me) since more ligand symmetries are tested. 

3. n_symmetries:2 and n_conformers:2:
```python
out = build_complex({'core':{'metal':'Fe','coreCN':6},
                     'ligands':['water']*3+['chloride','bromide','iodide'],
                     'parameters':{'n_symmetries':2,'n_conformers':2,'skip_duplicate_tests':True,'relax':False}
                    })
view_structures(out)
```

This should produce 8 structures now - two for each metal center symmetry: octahedral, pentagonal_pyramidal, trigonal_prismatic, and hexagonal planar. Note that if you remove the 'skip_duplicate_tests' and 'relax':False keywords, Architector will give you fewer structures than this maximum of 8 since Architector will remove some structures that 1. will be unstable at the GFN2-xTB level of theory(particularly the planar geometry) or 2. After GFN2-xTB relaxation converge to nearly identical geometries.


## d. Still nothing

Problem: Still nothing is being produced.

Solutions:
1. Play with sanity check cutoffs. You can turn off final an intermediate sanity checks with "full_sanity_checks":False, and "assemble_sanity_checks":False, respectively. You can also tune the actual cutoff values as well : "full_graph_sanity_cutoff":1.7 - can be tuned higher to allow for looser constraints, "full_smallest_dist_cutoff":0.55 - can be tuned lower for closer interatomic distances, and "full_min_dist_cutoff":3.5 - can be tuned higher to allow for more isolated atoms.
2. Metal van-der-waals radii ("vdwrad_metal") and covalent radii ("covrad_metal") can be manually tuned in cases where a very distinct oxiation state is used. 
3. You might have ligands with trailing carbon chains that tend to overlap when being tile to the metal center surface. For example, the TODGA ligand can have this issue. Try setting "ff_preopt":True - this will generate the full complex, relax it with UFF - and only then perform the final relaxation method (usually GFN2-xTB). The UFF pre-optimization dramatically increases the chance for successful XTB relaxation in tests, but does introduce more bias into the generated structures.
3. A last-dich effort can be made to simply force Architector to return basically anything with "force_generation":True. This can be useful to determine if something is going wrong internally in Architector. It is not recommended to use this option for production (multiple-structures runs).

# 2 - Only a couple structures were produced? I wanted a bunch!

This is common when looking at smaller homogenous coordination spheres (e.g. Water-coordiantion spheres)

## a. Similar small ligands.

Problem: Only one or two structures produced when many desired.

Solution: Often this results from the complexes converging to very similar structures near the metal center - which architector will remove by default with duplicate checks. To resolve this I will usually turn off the duplicate checks "skip_duplicate_checks":True. 

## b. Only 1 or 2 unique solutions exist.

Problem: With rigid ligands such as bipyridine or terpyradine, multiple unique structures are often unfeasible.

Solution: None! Possibly look at functionalizations that result in more distinct chemistries.

## c. Architector only samples a specified number of unique symmetries.

Problem: By default architector will only check 10 different symmetries and relax only 1 for output. 

Solution: Increase "n_conformers" to relax additional metal center symmetries at each core geometry. See expalanation of n_symmetries and n_conformers above under 1(c). With increasing n_conformers you will increase sampling of generated molecules with different ligand symmetries in the output. You could also increase n_symmetries - but playing with both of these parameters can result in longer complex generation times due to additional overhead from relaxing all conformers.

In many cases playing with these parameters is an easy way to get more sampling required to find a "sane" conformer.

Any/all of these debugging tips can help give more structures as output - whether these are "good" remains up to the users to determine!

# 3 - I wanted cis-actinyl oxos.

This is a more fringe issue for many. By default if you run generation with an Actinide in the (V) or (VI) oxidation state Architector will force the OXO-groups to be trans since this is the much more commonly-observed configuration for actinyl groups. 

If you'd like the cis- structures as well simply run with "force_trans_oxos":False.

# More ideas: Please raise an issue on github or email the team to let us know if there's another issue that should be handled by Architector. We will update this document with more best "debugging" practices in addressing your issues!